# Examining image sizes of T1, MNI T1, T2*, and mask files

K. Garner, 2020

In [1]:
import nilearn.plotting as niplt # for debugging
import nilearn.image as img
import pandas as pd
import nibabel as nib
import os

In [2]:
# define subject paths
Basedir =  "/scratch/qbi/uqkgarn1/STRIWP1/"
T1fname = "derivatives/sub-01/anat/sub-01_desc-preproc_T1w.nii.gz"
T2s = ["derivatives/sub-01/ses-02/tSNR/sub-01_ses-02_task-learnAtt_acq-TR700_echo-1_space-T1w_desc-tSNR-resamp_bold.nii.gz",
       "derivatives/sub-01/ses-02/tSNR/sub-01_ses-02_task-learnAtt_acq-TR700_echo-2_space-T1w_desc-tSNR-resamp_bold.nii.gz",
       "derivatives/sub-01/ses-02/tSNR/sub-01_ses-02_task-learnAtt_acq-TR1510_space-T1w_desc-tSNR-resamp_bold.nii.gz",
       "derivatives/sub-01/ses-02/tSNR/sub-01_ses-02_task-learnAtt_acq-TR1920_space-T1w_desc-tSNR-resamp_bold.nii.gz"]
MNI = "derivatives/sub-01/anat/sub-01_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz"
Kmask = "derivatives/roi-masks/Keuken/LCN.nii.gz"
RFLmask = "derivatives/roi-masks/RFL/ca_mni.nii.gz"
T1fname = os.path.join(Basedir, T1fname)
MNIfname = os.path.join(Basedir, MNI)
Kfname = os.path.join(Basedir, Kmask)
RFLfname = os.path.join(Basedir, RFLmask)
#subMask = 

## check details re: the T1

In [3]:
T1 = nib.load(T1fname)
T1.shape

# T1.affine.shape
T1hdr = T1.header
T1hdr.get_zooms()

(0.75, 0.75, 0.75)

# comparing affine information for T2 resampled and T2 mean bold

See https://nipy.org/nibabel/nifti_images.html
The affine relates the voxel coordinates to world coordinates in RAS+ space (whatever that is).
Nibabel has an algorithm to pick whether it will use the sform, qform ro fall-back header affine

a) are there any differences in the affine info for the resampled and the mean bold images? 
b) how does spm choose which affine to use?
c) can I set an affine for spm?

__spm info on the nibabel page__
This is the affine of last resort, constructed only from the pixdim voxel sizes. The NIfTI specification says that this should set the first voxel in the image as [0, 0, 0] in world coordinates, but we nibabblers follow SPM in preferring to set the central voxel to have [0, 0, 0] world coordinate. The NIfTI spec also implies that the image should be assumed to be in RAS+ voxel orientation for this affine (see Coordinate systems and affines). Again like SPM, we prefer to assume LAS+ voxel orientation by default.

In [46]:
# has been averaged but not resampled
T2mu = nib.load(os.path.join(Basedir, 'derivatives/sub-01/ses-02/func/sub-01_ses-02_task-learnAtt_acq-TR700_echo-1_space-T1w_desc-mean_bold.nii.gz'))
# has been used by afni but not resampled
# T2tSNR = nib.load(os.path.join(Basedir, 'derivatives/sub-01/ses-02/func/sub-01_ses-02_task-learnAtt_acq-TR1510_space-T1w_desc-mean_bold.nii.gz')       
# has been resampled by afni
T2tSNR_resamp = nib.load(os.path.join(Basedir, 'derivatives/sub-01/ses-02/tSNR/sub-01_ses-02_task-learnAtt_acq-TR700_echo-2_space-T1w_desc-tSNR-resamp_bold.nii.gz'))

first get the sform info for each file type

In [47]:
T2mu_hdr = T2mu.header
T2tSNR_resamp_hdr = T2tSNR_resamp.header

In [48]:
T2mu_hdr.get_base_affine()

array([[ -2.,   0.,   0.,  70.],
       [  0.,   2.,   0., -94.],
       [  0.,   0.,   2., -68.],
       [  0.,   0.,   0.,   1.]])

In [20]:
print(T2mu_hdr['sform_code'])

2


In [ ]:
#T2mu_hdr = #

In [28]:
T2tSNR_resamp_hdr.get_base_affine()

array([[ -0.75 ,   0.   ,   0.   ,  70.5  ],
       [  0.   ,   0.75 ,   0.   , -94.5  ],
       [  0.   ,   0.   ,   0.75 , -68.625],
       [  0.   ,   0.   ,   0.   ,   1.   ]])

In [21]:
print(T2tSNR_resamp_hdr['sform_code'])

3


It looks like AFNI have changed the code on the affine during resample, to put into talaraich space, given that the sform code is now 3 (aligned + Talairach), instead of 2 (aligned to an unknown space), which is what it should be. I am assuming that AFNI detected the master as being a Talaraich.

So I will rerun the tSNR code WITHOUT resampling, and check whether or not I get a RAS code of 2 for the output. I will then try using this in SPM, once it has the 'correct' RAS code.

In [23]:
T2_tSNR = nib.load(os.path.join(Basedir, 'derivatives/sub-01/ses-02/tSNR/sub-01_ses-02_task-learnAtt_acq-TR700_echo-1_space-T1w_desc-tSNR_bold.nii.gz'))

In [25]:
print(T2tSNR_hdr['sform_code'])

3


Weird - so its not the resample function that has done it. I'll open the residual file to see if that is what happened

In [26]:
T2_resid = nib.load(os.path.join(Basedir, 'derivatives/glm/sub-01/ses-02/func/sub-01_ses-02_task-learnAtt_acq-TR700_echo-1_space-T1w_desc-resid_bold.nii.gz'))
T2_resid_hdr = T2_resid.header

In [27]:
print(T2_resid_hdr['sform_code'])

3


OK, so the mean bold, which has not been passed through the AFNI glm function, has the correct code. I'll just check the pre-processed bold that is fed into the AFNI glm function, and if that has code 2, then I know its the GLM function. If thats the case, I'll see if I can switch the header information for the T2 data, prior to the tSNR comp/resampling.


In [29]:
T2_preproc = nib.load(os.path.join(Basedir, 'derivatives/sub-01/ses-02/func/sub-01_ses-02_task-learnAtt_acq-TR700_echo-1_space-T1w_desc-preproc_bold.nii.gz'))

In [30]:
T2_preproc_hdr = T2_preproc.header

In [32]:
print(T2_preproc_hdr['sform_code'])

2


Alriiiiight - so it looks like the afni glm function I used at the beginning is to blame. The question is whether it applied a transformation when it changed the code (maybe it was to automatically apply a brain mask). To check the spaces, I'll compare the T2_preproc_hdr affine intel and the T2_resid affine intel - below (and will also compare to the T2_mu stuff above)

In [33]:
T2_preproc_hdr.get_base_affine()

array([[ -2.,   0.,   0.,  70.],
       [  0.,   2.,   0., -94.],
       [  0.,   0.,   2., -68.],
       [  0.,   0.,   0.,   1.]])

In [35]:
T2_resid_hdr.get_base_affine()

array([[ -2.,   0.,   0.,  70.],
       [  0.,   2.,   0., -94.],
       [  0.,   0.,   2., -68.],
       [  0.,   0.,   0.,   1.]])

This suggests that the base affine information is the same across the two files. Now I'll check the sform affine

In [36]:
print(T2_preproc_hdr.get_sform())

[[  2.           0.           0.         -67.32543182]
 [  0.           2.           0.         -73.50365448]
 [  0.           0.           2.         -79.72452545]
 [  0.           0.           0.           1.        ]]


In [37]:
print(T2_resid_hdr.get_sform())

[[  2.          -0.          -0.         -67.32543182]
 [ -0.           2.          -0.         -73.50365448]
 [  0.           0.           2.         -79.72452545]
 [  0.           0.           0.           1.        ]]


In [39]:
print(T2tSNR_hdr.get_sform())

[[  2.          -0.          -0.         -67.32543182]
 [ -0.           2.          -0.         -73.50365448]
 [  0.           0.           2.         -79.72452545]
 [  0.           0.           0.           1.        ]]


Apart from the changing zeros to minuses, it looks the same across the two filetypes. I am going to now just check the dimensions of the data between the two files to check they are the same. If so, I think that resetting the RAS code is a good experimental test for the spm_get_data function. In fact, it is highly unlikely that any tranform was actually applied, because the tSNR files line up over the participant T1s pretty well.

In [38]:
T2_preproc.shape

(71, 95, 69, 918)

In [40]:
T2_resid.shape

(71, 95, 69, 812)

In [42]:
T2_tSNR.shape 

(71, 95, 69)

In [49]:
T2mu.shape

(71, 95, 69)

They are all lining up, so I will seek to change the RAS code of the TSNR shape file. I'll then download that and apply the SPM get data function to it.

Hmmm, so if I look at the resampled data, the xyz (offset?) is approx the same (i.e. the values in the final column)

I think I need to look into the spm_get_data file to understand how these numbers will affect my output.
euw - there be monsters there

I will compare the data I get out when a) the affine is the same, but the RAS is different, then
b) I'll check the output

Checks to make - 1) apply spm_get_data to the mean BOLD data - then change the RAS code to see if I get different values.

Matlab test showed that the sform_code (RAS) does not make a difference for spm_get_data - so I will next try extracting the data using the new tSNR images that I made earlier today (I will download the first and try it in matlab with spm).

# quickly look at the image to check its ok

In [ ]:
meanT2_700 = img.mean_img(T2_700_resamp)

# overlaying functional and anatomical images

Here I will plot the participant T1 image, with the resampled T2 over the top

In [ ]:
fig = niplt.plot_epi(meanT2_700, alpha=0.2)
fig.add_edges(T1, colors='r')

In [ ]:
from IPython.display import SVG

In [ ]:
SVG("/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-02/figures/sub-01_ses-02_task-learnAtt_acq-TR1920_desc-bbregister_bold.svg")

# compute mean images for download for examination in fsleyes

(have done sub 01, ses 02 already)